# Imports

In [17]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import *
import scipy

import warnings
warnings.filterwarnings("ignore")

In [18]:
plt.style.use('ggplot')

## Loading Data

In [22]:
raw_df = pd.read_csv(r"pollution_us_2000_2016.csv")

In [23]:
df = raw_df.copy()

In [24]:
df

,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,1.145833,4.200,21,NaN
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,0.878947,2.200,23,25.0
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,1.145833,4.200,21,NaN
3,3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,0.878947,2.200,23,25.0
4,4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,Parts per billion,1.958333,3.0,22,4.0,Parts per million,0.850000,1.600,23,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1746656,24599,56,21,100,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-30,Parts per billion,...,Parts per billion,0.000000,0.0,2,NaN,Parts per million,0.091667,0.100,2,1.0
1746657,24600,56,21,100,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-31,Parts per billion,...,Parts per billion,-0.022727,0.0,0,0.0,Parts per million,0.067714,0.127,0,NaN
1746658,24601,56,21,100,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-31,Parts per billion,...,Parts per billion,-0.022727,0.0,0,0.0,Parts per million,0.100000,0.100,0,1.0
1746659,24602,56,21,100,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-31,Parts per billion,...,Parts per billion,0.000000,0.0,5,NaN,Parts per million,0.067714,0.127,0,NaN


### EDA

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1746661 entries, 0 to 1746660
Data columns (total 29 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Unnamed: 0         int64  
 1   State Code         int64  
 2   County Code        int64  
 3   Site Num           int64  
 4   Address            object 
 5   State              object 
 6   County             object 
 7   City               object 
 8   Date Local         object 
 9   NO2 Units          object 
 10  NO2 Mean           float64
 11  NO2 1st Max Value  float64
 12  NO2 1st Max Hour   int64  
 13  NO2 AQI            int64  
 14  O3 Units           object 
 15  O3 Mean            float64
 16  O3 1st Max Value   float64
 17  O3 1st Max Hour    int64  
 18  O3 AQI             int64  
 19  SO2 Units          object 
 20  SO2 Mean           float64
 21  SO2 1st Max Value  float64
 22  SO2 1st Max Hour   int64  
 23  SO2 AQI            float64
 24  CO Units           object 
 25  CO Mean           

In [26]:
df.describe(include="all")

,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
count,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1746661,1746661,1746661,1746661,1746661,1746661,...,1746661,1.746661e+06,1.746661e+06,1.746661e+06,873754.000000,1746661,1.746661e+06,1.746661e+06,1.746661e+06,873338.000000
unique,NaN,NaN,NaN,NaN,204,47,133,144,5996,1,...,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,PIKE AVE AT RIVER ROAD,California,Los Angeles,Not in a city,2002-06-10,Parts per billion,...,Parts per billion,NaN,NaN,NaN,NaN,Parts per million,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,35332,576142,93381,138411,640,1746661,...,1746661,NaN,NaN,NaN,NaN,1746661,NaN,NaN,NaN,NaN
mean,5.471414e+04,2.230907e+01,7.169381e+01,1.118214e+03,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.870364e+00,4.492185e+00,9.664906e+00,7.115945,NaN,3.682177e-01,6.201067e-01,7.875026e+00,5.996595
std,3.372908e+04,1.725621e+01,7.948023e+01,2.003103e+03,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.760435e+00,7.679866e+00,6.731228e+00,11.937473,NaN,3.140231e-01,6.439361e-01,7.978844e+00,5.851836
min,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-2.000000e+00,-2.000000e+00,0.000000e+00,0.000000,NaN,-4.375000e-01,-4.000000e-01,0.000000e+00,0.000000
25%,2.575300e+04,6.000000e+00,1.700000e+01,9.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.565220e-01,8.000000e-01,5.000000e+00,1.000000,NaN,1.834580e-01,2.920000e-01,0.000000e+00,2.000000
50%,5.304500e+04,1.700000e+01,5.900000e+01,6.000000e+01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9.875000e-01,2.000000e+00,8.000000e+00,3.000000,NaN,2.926250e-01,4.000000e-01,6.000000e+00,5.000000
75%,8.033600e+04,4.000000e+01,9.700000e+01,1.039000e+03,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.325000e+00,5.000000e+00,1.400000e+01,9.000000,NaN,4.666670e-01,8.000000e-01,1.300000e+01,8.000000


In [27]:
object_columns = df.select_dtypes(include='object')

In [28]:
object_columns

,Address,State,County,City,Date Local,NO2 Units,O3 Units,SO2 Units,CO Units
0,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,Parts per million,Parts per billion,Parts per million
1,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,Parts per million,Parts per billion,Parts per million
2,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,Parts per million,Parts per billion,Parts per million
3,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,Parts per million,Parts per billion,Parts per million
4,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,Parts per million,Parts per billion,Parts per million
...,...,...,...,...,...,...,...,...,...
1746656,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-30,Parts per billion,Parts per million,Parts per billion,Parts per million
1746657,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-31,Parts per billion,Parts per million,Parts per billion,Parts per million
1746658,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-31,Parts per billion,Parts per million,Parts per billion,Parts per million
1746659,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-31,Parts per billion,Parts per million,Parts per billion,Parts per million


In [29]:
object_columns.describe()

,Address,State,County,City,Date Local,NO2 Units,O3 Units,SO2 Units,CO Units
count,1746661,1746661,1746661,1746661,1746661,1746661,1746661,1746661,1746661
unique,204,47,133,144,5996,1,1,1,1
top,PIKE AVE AT RIVER ROAD,California,Los Angeles,Not in a city,2002-06-10,Parts per billion,Parts per million,Parts per billion,Parts per million
freq,35332,576142,93381,138411,640,1746661,1746661,1746661,1746661


In [30]:
object_columns_list = list(object_columns.columns)
object_columns_list

['Address',
 'State',
 'County',
 'City',
 'Date Local',
 'NO2 Units',
 'O3 Units',
 'SO2 Units',
 'CO Units']

In [31]:
non_object_columns = df.select_dtypes(exclude='object')

In [32]:
non_object_columns.describe()

,Unnamed: 0,State Code,County Code,Site Num,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
count,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,873754.000000,1.746661e+06,1.746661e+06,1.746661e+06,873338.000000
mean,5.471414e+04,2.230907e+01,7.169381e+01,1.118214e+03,1.282193e+01,2.541485e+01,1.173102e+01,2.389822e+01,2.612485e-02,3.920331e-02,1.017053e+01,3.605012e+01,1.870364e+00,4.492185e+00,9.664906e+00,7.115945,3.682177e-01,6.201067e-01,7.875026e+00,5.996595
std,3.372908e+04,1.725621e+01,7.948023e+01,2.003103e+03,9.504814e+00,1.599963e+01,7.877501e+00,1.516280e+01,1.136974e-02,1.534362e-02,4.003144e+00,1.978042e+01,2.760435e+00,7.679866e+00,6.731228e+00,11.937473,3.140231e-01,6.439361e-01,7.978844e+00,5.851836
min,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-2.000000e+00,-2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.000000e+00,-2.000000e+00,0.000000e+00,0.000000,-4.375000e-01,-4.000000e-01,0.000000e+00,0.000000
25%,2.575300e+04,6.000000e+00,1.700000e+01,9.000000e+00,5.750000e+00,1.300000e+01,5.000000e+00,1.200000e+01,1.787500e-02,2.900000e-02,9.000000e+00,2.500000e+01,2.565220e-01,8.000000e-01,5.000000e+00,1.000000,1.834580e-01,2.920000e-01,0.000000e+00,2.000000
50%,5.304500e+04,1.700000e+01,5.900000e+01,6.000000e+01,1.073913e+01,2.400000e+01,9.000000e+00,2.300000e+01,2.587500e-02,3.800000e-02,1.000000e+01,3.300000e+01,9.875000e-01,2.000000e+00,8.000000e+00,3.000000,2.926250e-01,4.000000e-01,6.000000e+00,5.000000
75%,8.033600e+04,4.000000e+01,9.700000e+01,1.039000e+03,1.771364e+01,3.570000e+01,2.000000e+01,3.300000e+01,3.391700e-02,4.800000e-02,1.100000e+01,4.200000e+01,2.325000e+00,5.000000e+00,1.400000e+01,9.000000,4.666670e-01,8.000000e-01,1.300000e+01,8.000000
max,1.345750e+05,8.000000e+01,6.500000e+02,9.997000e+03,1.395417e+02,2.670000e+02,2.300000e+01,1.320000e+02,9.508300e-02,1.410000e-01,2.300000e+01,2.180000e+02,3.216250e+02,3.510000e+02,2.300000e+01,200.000000,7.508333e+00,1.990000e+01,2.300000e+01,201.000000


In [33]:
non_object_columns_list = list(non_object_columns.columns)
non_object_columns_list

['Unnamed: 0',
 'State Code',
 'County Code',
 'Site Num',
 'NO2 Mean',
 'NO2 1st Max Value',
 'NO2 1st Max Hour',
 'NO2 AQI',
 'O3 Mean',
 'O3 1st Max Value',
 'O3 1st Max Hour',
 'O3 AQI',
 'SO2 Mean',
 'SO2 1st Max Value',
 'SO2 1st Max Hour',
 'SO2 AQI',
 'CO Mean',
 'CO 1st Max Value',
 'CO 1st Max Hour',
 'CO AQI']

#### Removing unnecessary columns

In [34]:
columns_to_be_removed = ["Unnamed: 0",
                         'Address',
                         'NO2 Units', 
                         'O3 Units', 
                         'SO2 Units', 
                         'CO Units']

**Reason for selecting these columns:**
- `Unnamed: 0` : Contains random numbers. (Unnecessary data)
- `Address` : Specific address of the sensors. (Address is already split into city, county and state)
- `NO2 Units`, `O3 Units`, `SO2 Units`, `CO Units` : Units for measuring AQI. (Created a common unit column)

In [36]:
treated_df = df.drop(columns_to_be_removed, axis=1)

In [37]:
treated_df

,State Code,County Code,Site Num,State,County,City,Date Local,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,...,O3 1st Max Hour,O3 AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,4,13,3002,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,19,...,10,34,3.000000,9.0,21,13.0,1.145833,4.200,21,NaN
1,4,13,3002,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,19,...,10,34,3.000000,9.0,21,13.0,0.878947,2.200,23,25.0
2,4,13,3002,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,19,...,10,34,2.975000,6.6,23,NaN,1.145833,4.200,21,NaN
3,4,13,3002,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,19,...,10,34,2.975000,6.6,23,NaN,0.878947,2.200,23,25.0
4,4,13,3002,Arizona,Maricopa,Phoenix,2000-01-02,22.958333,36.0,19,...,10,27,1.958333,3.0,22,4.0,0.850000,1.600,23,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1746656,56,21,100,Wyoming,Laramie,Not in a city,2016-03-30,1.083333,1.6,9,...,18,44,0.000000,0.0,2,NaN,0.091667,0.100,2,1.0
1746657,56,21,100,Wyoming,Laramie,Not in a city,2016-03-31,0.939130,1.3,5,...,9,44,-0.022727,0.0,0,0.0,0.067714,0.127,0,NaN
1746658,56,21,100,Wyoming,Laramie,Not in a city,2016-03-31,0.939130,1.3,5,...,9,44,-0.022727,0.0,0,0.0,0.100000,0.100,0,1.0
1746659,56,21,100,Wyoming,Laramie,Not in a city,2016-03-31,0.939130,1.3,5,...,9,44,0.000000,0.0,5,NaN,0.067714,0.127,0,NaN


In [38]:
treated_df["unit_of_measurement"] = "Parts per million"

Unique States 

In [39]:
len(treated_df.State.unique())

47

In [40]:
treated_df.State.unique()

array(['Arizona', 'California', 'Colorado', 'District Of Columbia',
       'Florida', 'Illinois', 'Indiana', 'Kansas', 'Kentucky',
       'Louisiana', 'Michigan', 'Missouri', 'New Jersey', 'New York',
       'North Carolina', 'Oklahoma', 'Pennsylvania', 'Texas', 'Virginia',
       'Massachusetts', 'Nevada', 'New Hampshire', 'Tennessee',
       'South Carolina', 'Connecticut', 'Iowa', 'Maine', 'Maryland',
       'Wisconsin', 'Country Of Mexico', 'Arkansas', 'Oregon', 'Wyoming',
       'North Dakota', 'Idaho', 'Ohio', 'Georgia', 'Delaware', 'Hawaii',
       'Minnesota', 'New Mexico', 'Rhode Island', 'South Dakota', 'Utah',
       'Alabama', 'Washington', 'Alaska'], dtype=object)

In [41]:
len(treated_df["State Code"].unique())

47

### Check AIQs

In [46]:
df_AIQ = treated_df[['State','Date Local','NO2 AQI','O3 AQI','SO2 AQI','CO AQI']]

In [47]:
df_AIQ

,State,Date Local,NO2 AQI,O3 AQI,SO2 AQI,CO AQI
0,Arizona,2000-01-01,46,34,13.0,NaN
1,Arizona,2000-01-01,46,34,13.0,25.0
2,Arizona,2000-01-01,46,34,NaN,NaN
3,Arizona,2000-01-01,46,34,NaN,25.0
4,Arizona,2000-01-02,34,27,4.0,NaN
...,...,...,...,...,...,...
1746656,Wyoming,2016-03-30,1,44,NaN,1.0
1746657,Wyoming,2016-03-31,1,44,0.0,NaN
1746658,Wyoming,2016-03-31,1,44,0.0,1.0
1746659,Wyoming,2016-03-31,1,44,NaN,NaN


In [48]:
df_AIQ.isnull().sum()

State              0
Date Local         0
NO2 AQI            0
O3 AQI             0
SO2 AQI       872907
CO AQI        873323
dtype: int64

In [52]:
df_AIQ["CO AQI"].isnull().sum()

873323

In [55]:
def get_amount_of_null_values(dataframe: pd.DataFrame):
    row_count = len(dataframe)
    print(f"Number of rows present in the dataset: {row_count}")
    columns_list = list(dataframe.columns)
    for i in columns_list:
        null_value_count = dataframe[i].isnull().sum()
        if null_value_count != 0:
            print(f"Column name: {i}, null values present: {null_value_count}, {(null_value_count/row_count)*100:.2f}% of total data.")

In [56]:
get_amount_of_null_values(df_AIQ)

Number of rows present in the dataset: 1746661
Column name: SO2 AQI, null values present: 872907, 49.98% of total data.
Column name: CO AQI, null values present: 873323, 50.00% of total data.


Since the presence of null values if very high, we can't drop all the null values. We will use some imputing method later in the notebook.

Dropping null values

In [44]:
# df_AIQ.dropna(inplace=True)

In [57]:
df_AIQ

,State,Date Local,NO2 AQI,O3 AQI,SO2 AQI,CO AQI
0,Arizona,2000-01-01,46,34,13.0,NaN
1,Arizona,2000-01-01,46,34,13.0,25.0
2,Arizona,2000-01-01,46,34,NaN,NaN
3,Arizona,2000-01-01,46,34,NaN,25.0
4,Arizona,2000-01-02,34,27,4.0,NaN
...,...,...,...,...,...,...
1746656,Wyoming,2016-03-30,1,44,NaN,1.0
1746657,Wyoming,2016-03-31,1,44,0.0,NaN
1746658,Wyoming,2016-03-31,1,44,0.0,1.0
1746659,Wyoming,2016-03-31,1,44,NaN,NaN


In [58]:
df_AIQ['Date Local'] = pd.to_datetime(df_AIQ['Date Local'],format='%Y-%m-%d')  # Change date from string to date value


In [59]:
df_AIQ

,State,Date Local,NO2 AQI,O3 AQI,SO2 AQI,CO AQI
0,Arizona,2000-01-01,46,34,13.0,NaN
1,Arizona,2000-01-01,46,34,13.0,25.0
2,Arizona,2000-01-01,46,34,NaN,NaN
3,Arizona,2000-01-01,46,34,NaN,25.0
4,Arizona,2000-01-02,34,27,4.0,NaN
...,...,...,...,...,...,...
1746656,Wyoming,2016-03-30,1,44,NaN,1.0
1746657,Wyoming,2016-03-31,1,44,0.0,NaN
1746658,Wyoming,2016-03-31,1,44,0.0,1.0
1746659,Wyoming,2016-03-31,1,44,NaN,NaN


In [ ]:
# NO2 AQI
plt.subplot(221)
pollNO2 = pollStGrouped['NO2 AQI']
pollNO2Top = pollNO2.mean().nlargest(4).index
for i in range(len(pollNO2Top)):
    pollNO2.get_group(pollNO2Top[i]).groupby(pd.Grouper(level='Date Local',freq='M')).mean().plot()
plt.legend(pollNO2Top,loc=3,fontsize='small')
plt.title('NO2 AQI')

  ### Feature Engineering

Extract features day, month and year of the data

In [7]:
def get_day_month_year():
    

IndentationError: expected an indented block (496633006.py, line 2)

### Prevention on data leakage in time series data

## ML Modeling

### Forecasting

### Clasification